In [68]:
from lattedb.quarksmear.models import GaugeCovariantGaussian as quarksmear_GaugeCovariantGaussian
from lattedb.fermionaction.models import Hisq as fermionaction_Hisq
from lattedb.gaugeaction.models import LuescherWeisz as gaugeaction_LuescherWeisz
from lattedb.fermionaction.models import MobiusDW as fermionaction_MobiusDW
from lattedb.gaugeconfig.models import Nf211 as gaugeconfig_Nf211
from lattedb.quarksmear.models import Point as quarksmear_Point
from lattedb.linksmear.models import Unsmeared as linksmear_Unsmeared
from lattedb.linksmear.models import WilsonFlow as linksmear_WilsonFlow
from lattedb.propagator.models import OneToAll as propagator_OneToAll

In [69]:
import sys
sys.path.append('../../nucleon_elastic_FF/scripts/area51_files')
sys.path.append('../../nucleon_elastic_FF/scripts')
import sources
import utils

In [70]:
import a09m310 as a51
p = a51.mpirun_params("summit")
p["STREAM"] = "e"

In [71]:
cfgs_run, p['srcs'] = utils.parse_cfg_src_argument([p["cfg_i"], p["cfg_f"], p["cfg_d"]],'',p)

In [72]:
gaugeconfigs = gaugeconfig_Nf211.objects.filter(short_tag=p["ENS_ABBR"])

In [73]:
linksmear_tag = f"wflow{p['FLOW_TIME']}_n{p['FLOW_STEP']}"
fermionaction_linksmear, created = linksmear_WilsonFlow.objects.get_or_create(
	flowtime=p["FLOW_TIME"], # Flow time in lattice units
	flowstep=p["FLOW_STEP"], # Number of diffusion steps
	tag=linksmear_tag, # (Optional) User defined tag for easy searches
)
print(created)

smear_tag = f"wv_w{p['WF_S']}_n{p['WF_N']}"
gaussian, created = quarksmear_GaugeCovariantGaussian.objects.get_or_create(
	radius=p["WF_S"], # Smearing radius in lattice units
	step=p["WF_N"], # Number of smearing steps
	tag= smear_tag, # (Optional) User defined tag for easy searches
	description="Gaussian smearing.", # (Optional) Description of the quark smearing operator
)
print(created)

point, created = quarksmear_Point.objects.get_or_create(
	tag="", # (Optional) User defined tag for easy searches
	description="Dirac delta function.", # (Optional) Description of the quark smearing operator
)
print(created)
smearings = [('SS', gaussian), ('PS', point)]

False
False
False


In [1]:
quark_list = [("light", p["MV_L"]), ("strange", p["MV_S"])]
for quark in quark_list:
    fermion_tag = f"M5{p['M5']}_L5{p['L5']}_a{p['alpha5']}"
    fermionaction, created = fermionaction_MobiusDW.objects.get_or_create(
        quark_mass=quark[1], # Input quark mass
        quark_tag=quark[0], # Type of quark
        l5=p["L5"], # Length of 5th dimension
        m5=p["M5"], # 5th dimensional mass
        b5=p["B5"], # Mobius kernel parameter [a5 = b5 - c5, alpha5 * a5 =…
        c5=p["C5"], # Mobius kernal perameter
        linksmear=fermionaction_linksmear, # Foreign Key pointing to additional gauge `linksmear` outside of Monte Carlo.
        tag=fermion_tag, # (Optional) User defined tag for easy searches
    )
    print("fermionaction:", created)
    for smearing in smearings:
        for gaugeconfig in gaugeconfigs:
            for source in p['srcs'][gaugeconfig.config]:
                xyzt = sources.xyzt(source)
                prop_tag = f"prop_{smearing[0]}_mq{quark[1]}_{linksmear_tag}_{fermion_tag}_cfg_{gaugeconfig.config}_src{source}_{smear_tag}"
                propagator_onetoall, created = propagator_OneToAll.objects.update_or_create(
                    gaugeconfig=gaugeconfig, # Foreign Key pointing to specific \texttt{gaugeconfig} inverted on
                    fermionaction=fermionaction, # Foreign Key pointing to valence lattice `fermionaction`. This is the valence action.
                    origin_x=int(xyzt[0]), # x-coordinate origin location of the propagator
                    origin_y=int(xyzt[1]), # y-coordinate origin location of the propagator
                    origin_z=int(xyzt[2]), # z-coordinate origin location of the propagator
                    origin_t=int(xyzt[3]), # t-coordinate origin location of the propagator
                    sourcesmear=gaussian, # Foreign Key pointing to source `quarksmear`
                    sinksmear=smearing[1], # Foreign Key pointing to sink `quarksmear`
                    tag=prop_tag, # (Optional) User defined tag for easy searches
                )
                print(f"\r>>{created} {prop_tag}", end="")

NameError: name 'p' is not defined